In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from functools import partial
import joblib
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression

sys.path.append('../')

DIR = '/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/'
description = pd.read_csv(os.path.join(DIR,'data/HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv(os.path.join(DIR, 'data/application_test.csv'))
bureau = pd.read_csv(os.path.join(DIR, 'data/bureau.csv'))
bureau_balance = pd.read_csv(os.path.join(DIR, 'data/bureau_balance.csv'))

In [2]:
bureau_balance = bureau_balance.merge(bureau[['SK_ID_CURR', 'SK_ID_BUREAU']], on='SK_ID_BUREAU', how='right')
bureau_balance

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,SK_ID_CURR
0,5714462,NaN,NaN,215354
1,5714463,NaN,NaN,215354
2,5714464,NaN,NaN,215354
3,5714465,NaN,NaN,215354
4,5714466,NaN,NaN,215354
...,...,...,...,...
25121810,5057750,NaN,NaN,259355
25121811,5057754,NaN,NaN,100044
25121812,5057762,NaN,NaN,100044
25121813,5057770,NaN,NaN,246829


In [3]:
def _status_to_int(status):
    if status in ['X', 'C']:
        return 0
    if pd.isnull(status):
        return np.nan
    return int(status)

In [4]:
bureau_balance['bureau_balance_dpd_level'] = bureau_balance['STATUS'].apply(_status_to_int)
bureau_balance['bureau_balance_status_unknown'] = (bureau_balance['STATUS'] == 'X').astype(int)

In [5]:
#groupby = bureau_balance.groupby(['SK_ID_CURR'])
features_all = pd.DataFrame({'SK_ID_CURR': bureau_balance['SK_ID_CURR'].unique()})

In [6]:
PREVIOUS_APPLICATION_AGGREGATION_RECIPIES = []
for agg in ['sum', 'mean', 'max', 'std', 'skew']:
    for select in ['bureau_balance_dpd_level',
                   'bureau_balance_status_unknown'
                   ]:
        PREVIOUS_APPLICATION_AGGREGATION_RECIPIES.append((select, agg))
PREVIOUS_APPLICATION_AGGREGATION_RECIPIES = [(['SK_ID_CURR'], PREVIOUS_APPLICATION_AGGREGATION_RECIPIES)]

In [7]:
PREVIOUS_APPLICATION_AGGREGATION_RECIPIES=[(['SK_ID_CURR'],
  [('bureau_balance_dpd_level', 'sum'),
   ('bureau_balance_dpd_level', 'mean'),
   ('bureau_balance_dpd_level', 'max'),
   #('bureau_balance_dpd_level', 'kurt'),
   ('bureau_balance_status_unknown', 'sum'),
   ('bureau_balance_status_unknown', 'mean')
   ])]

In [8]:
prev_applications = bureau_balance.sort_values(['SK_ID_CURR', 'MONTHS_BALANCE'])
groupby_aggregate_names = []
number_of_applications = [1, 3, 5]
#number = 1
for number in number_of_applications:
    for groupby_cols, specs in tqdm(PREVIOUS_APPLICATION_AGGREGATION_RECIPIES):
        group_object = prev_applications.groupby(groupby_cols).tail(number)
        group_object = group_object.groupby(groupby_cols)
        # prev_applications_tail = prev_app_sorted.groupby('SK_ID_CURR').tail(number)
        # tail_groupby = prev_applications_tail.groupby('SK_ID_CURR')
        for select, agg in tqdm(specs):
            groupby_aggregate_name = '{}_{}_{}_{}_{}'.format('_'.join(groupby_cols), agg, select,'last', number)
            features_all = features_all.merge(group_object[select]
                                .agg(agg)
                                .reset_index()
                                .rename(index=str,
                                        columns={select: groupby_aggregate_name})
                                [groupby_cols + [groupby_aggregate_name]],
                                on=groupby_cols,
                                how='left')
            groupby_aggregate_names.append(groupby_aggregate_name)

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_82911/772550620.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for groupby_cols, specs in tqdm(PREVIOUS_APPLICATION_AGGREGATION_RECIPIES):


  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_82911/772550620.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for select, agg in tqdm(specs):


  0%|          | 0/5 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_82911/772550620.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for groupby_cols, specs in tqdm(PREVIOUS_APPLICATION_AGGREGATION_RECIPIES):


...

  0%|          | 0/5 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_82911/772550620.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for groupby_cols, specs in tqdm(PREVIOUS_APPLICATION_AGGREGATION_RECIPIES):


  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_82911/772550620.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for select, agg in tqdm(specs):


  0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
features_all

,SK_ID_CURR,SK_ID_CURR_sum_bureau_balance_dpd_level_last_1,SK_ID_CURR_mean_bureau_balance_dpd_level_last_1,SK_ID_CURR_max_bureau_balance_dpd_level_last_1,SK_ID_CURR_sum_bureau_balance_status_unknown_last_1,SK_ID_CURR_mean_bureau_balance_status_unknown_last_1,SK_ID_CURR_sum_bureau_balance_dpd_level_last_3,SK_ID_CURR_mean_bureau_balance_dpd_level_last_3,SK_ID_CURR_max_bureau_balance_dpd_level_last_3,SK_ID_CURR_sum_bureau_balance_status_unknown_last_3,SK_ID_CURR_mean_bureau_balance_status_unknown_last_3,SK_ID_CURR_sum_bureau_balance_dpd_level_last_5,SK_ID_CURR_mean_bureau_balance_dpd_level_last_5,SK_ID_CURR_max_bureau_balance_dpd_level_last_5,SK_ID_CURR_sum_bureau_balance_status_unknown_last_5,SK_ID_CURR_mean_bureau_balance_status_unknown_last_5
0,215354,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0
1,162297,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0
2,402440,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0
3,238881,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0
4,222183,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,207190,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
305807,324956,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
305808,448157,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0,0.0,NaN,NaN,0,0.0
305809,345866,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0


In [10]:
features_all.to_parquet('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/features_test/bureau_balance.parquet')